In [1]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from dist_yolo.dist_yolo import DistYOLO
import cv2

In [2]:
dist_yolo = DistYOLO()
dist_yolo.load_model('./dist_yolo/trained_final (2).h5')
# image = Image.open(input_image_path)
# pred = dist_yolo.detect(image)
# print(pred)

In [5]:
input_image_path = r"./images/input1.png"
img = Image.open(input_image_path)
# result = dist_yolo.detect_image(img)
# result.show()
result = dist_yolo.detect(img)
dets = list(zip(result[0], result[1], result[2], result[3]))

1/1 [==============================] - 1s 739ms/step


In [6]:
# draw bounding box
output_image = Image.open(input_image_path)
draw = ImageDraw.Draw(output_image)
for p in dets:
    if p[1] != 1 and p[1] != 2 and p[1] != 3:
        continue
    
    draw.rectangle(((p[0][0], p[0][1]), (p[0][2], p[0][3])))
    draw.text(((p[0][0] + p[0][2]) / 2, (p[0][1] + p[0][3]) / 2), f'{p[3]}')

# output_image.save('output.jpg', 'JPEG')
output_image.show()

## Draw Bounding Box to Raw Dist-YOLO

In [11]:
file_name = "video/video3.mp4"
output_file_name = "output/video3_3_output.mp4"

In [12]:
import cv2
import json

# Draw bounding box to video
cap = cv2.VideoCapture(file_name)
with open('output/dist_yolo_output_video3_3.json') as f:
    dyo = json.load(f)['final_frames_detections']

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = (
            int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
out = cv2.VideoWriter()
out.open(output_file_name, fourcc, fps, (width, height), True)

proc_frames = 0
try:
    while proc_frames < len(dyo):
        ret, frame = cap.read()
        if not ret:
            break

        im = frame
        # Loop through list (if empty this will be skipped) and overlay green bboxes
        for i in dyo[proc_frames]:
            cv2.rectangle(im, (i[0], i[1]), (i[2], i[3]), (0, 255, 0), 3)
            cv2.putText(im, f'{int(i[6])}', ((i[0] + i[2])//2, (i[1]+i[3])//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # write the frame
        out.write(im)

        proc_frames += 1
except Exception as error:
    # Release resources
    print(error)
    cap.release()
    out.release()

out.release()
cap.release()

## Kalman Tracking

In [49]:
from kalman_tracking.sort import Sort
import json
import cv2
import numpy as np

file_name = "video/video1.mp4"
output_file_name = "output/video1_3_output_tracked.mp4"
output_state_file_name = "output/video1_3_states.json"

sort = Sort(max_age=10, iou_threshold=0.3, zc=320)

cap = cv2.VideoCapture(file_name)
with open('output/dist_yolo_output_video1_3.json') as f:
    dyo = json.load(f)['final_frames_detections']

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = (
            int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
out = cv2.VideoWriter()
out.open(output_file_name, fourcc, fps, (width, height), True)

proc_frames = 0
states = []
color_mapping = {}
try:
    while proc_frames < len(dyo):
        ret, frame = cap.read()
        if not ret:
            break

        im = frame
        # Loop through list (if empty this will be skipped) and overlay green bboxes
        d = dyo[proc_frames]
        d_converted = []
        for x in d:
            d_converted.append([x[0], x[1], x[2], x[3], x[6]])
        d_converted = np.array(d_converted)

        # tracking
        if len(d_converted) > 0:
            xs, xrs = sort.update(d_converted)
        else:
            xs, xrs = sort.update()

        temp = []
        for (x, xr) in zip(xs, xrs):
            vehicle_id = int(x[-1])
            if vehicle_id not in color_mapping:
                bounding_rect = im[int(x[1]):int(x[3]), int(x[0]):int(x[2])]
                bgr_color = np.average(np.average(bounding_rect, axis=0), axis=0)

                hsv_color = cv2.cvtColor(np.array([[bgr_color]]).astype(np.uint8), cv2.COLOR_BGR2HSV)
                hsv_color[:, :, 1] = np.array(min(hsv_color[:, :, 1] * 1.5, 255)).astype(np.uint8)
                hsv_color[:, :, 2] *= np.array(min(hsv_color[:, :, 2] * 1, 255)).astype(np.uint8)
                bgr_color = cv2.cvtColor(hsv_color, cv2.COLOR_HSV2BGR)[0, 0]

                rgb_color = [int(bgr_color[2]), int(bgr_color[1]), int(bgr_color[0])]
                color_mapping[vehicle_id] = rgb_color

            temp.append(xr.tolist() + [int(x[-1]), color_mapping[vehicle_id]])
            cv2.rectangle(im, (int(x[0]), int(x[1])), (int(x[2]), int(x[3])), (0, 255, 0), 3)
            cv2.putText(im, f'{int(x[4])} ({int(xr[0])},{int(xr[1])},{int(xr[2])})', (int((x[0] + x[2])//2), int((x[1] + x[3])//2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        states.append(temp)

        # write the frame
        out.write(im)

        proc_frames += 1
except Exception as error:
    # Release resources
    cap.release()
    out.release()
    print(error.with_traceback())

out.release()
cap.release()

with open(output_state_file_name, 'w', encoding='utf-8') as f:
    json.dump({ "states": states }, f, ensure_ascii=False, indent=4)

c:\Bryan\TA\Integrated\kalman_tracking\sort.py:74: RuntimeWarning: invalid value encountered in sqrt
  w = np.sqrt(x[3] * x[4])


## Reconstruction

In [1]:
import numpy as np
from mygplvm.reconstruction import Reconstruction

sdfs = []
sdf_names = ['Bus', 'Jeep', 'Pickup', 'Sedan', 'Suv']
voxel_resolution = 64
dct_resolution = 25
for name in sdf_names:
    sdfs.append(np.load(f'./mygplvm/objs2/{name}_SDF_{voxel_resolution}x{voxel_resolution}x{voxel_resolution}.npy'))

rec = Reconstruction(voxel_resolution=voxel_resolution, dct_resolution=dct_resolution)
rec.fit_from_sdf(sdfs=sdfs)


         Current function value: 92231.596252
         Iterations: 3
         Function evaluations: 796
         Gradient evaluations: 56
time: 0.20706391334533691
alpha 1.172870663390303
beta 0.6048697071085747
gamma 0.9999992562935022


c:\ProgramData\anaconda3\envs\python310\lib\site-packages\scipy\optimize\_optimize.py:1698: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, **opts)


array([[ 51.15093994,   7.46380615],
       [-42.88323975,  12.87239742],
       [ 13.78208542,  -2.41257048],
       [-13.68172067, -10.06780717],
       [ -8.36767489,  -7.85581305]])

In [50]:
import pyrender
import matplotlib.pyplot as plt
from math import cos, sin, pi
import json
import cv2
import trimesh

output_file_name = 'output/video1_3_output_reconstructed.mp4'

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = int(30)
width, height = (
    int(1280),
    int(720)
)
out = cv2.VideoWriter()
out.open(output_file_name, fourcc, fps, (width, height), True)

with open('output/video1_3_states.json') as f:
    states = json.load(f)['states']

try:
    for i, state in enumerate(states):
        if (i == 10): break
        scene = pyrender.Scene(ambient_light=[0.6, 0.6, 0.6], bg_color=[1.0, 1.0, 1.0])
        state_multiplier = 6
        for vehicle in state:
            mc_mesh = rec.reconstruct_from_x([-13.68172067, -10.06780717])
            # for facet in mc_mesh.facets:
            #     mc_mesh.visual.face_colors[facet] = vehicle[10] + [255]
            mc_mesh.visual.vertex_colors = vehicle[10] + [255]

            mesh_pyrender = pyrender.Mesh.from_trimesh(mc_mesh, smooth=False)
            sudut = 0
            mesh_pose = np.array([
                [cos(sudut), 0, sin(sudut), vehicle[0] * state_multiplier],
                [0, 1, 0, 0],
                [-sin(sudut), 0, cos(sudut), -vehicle[2] * state_multiplier],
                [0, 0, 0, 1]
            ])
            scene.add(mesh_pyrender, pose=mesh_pose)

        camera_pose = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 62],
            [0, 0, 0, 1]
        ])
        light_pose = np.array([
            [1, 0, 0, 62],
            [0, 1, 0, 62],
            [0, 0, 1, 62],
            [0, 0, 0, 1]
        ])
        pl = pyrender.PointLight(color=[1.0, 1.0, 1.0], intensity=50000)
        pc = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1280/720)
        scene.add(pl, pose=light_pose)
        scene.add(pc, pose=camera_pose)

        r = pyrender.OffscreenRenderer(1280, 720)
        color, _ = r.render(scene)
        # plt.figure(figsize=(8,8)), plt.imshow(color)
        out.write(color)
except Exception as error:
    # Release resources
    out.release()
    print(error.with_traceback())

out.release()
    